In [55]:

import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(os.environ["CUDA_VISIBLE_DEVICES"])

0


In [56]:
from pathlib import Path
import ast

import numpy as np
import pandas as pd

In [57]:
def split_list(alist, startswith):
    split_start_idx = [i for i, s in enumerate(alist) if s.startswith(startswith)]
    split_end_idx = split_start_idx[1:] + [len(alist)]
    split_lines = [alist[i:j] for i, j in zip(split_start_idx, split_end_idx)]
    return split_lines

In [63]:
LOG_PATH = Path('/gaueko0/users/nmishra/multiling_fludetection/logs')
LOG_NUM = '9350'
file = open(LOG_PATH.joinpath(f'{LOG_NUM}.out'), 'r')
lines = file.readlines()
lines[:5]

['wandb artifacts and data saved in \n',
 '/gscratch3/users/nmishra/gridsearch_revisedcateg \n',
 'and rest in /gaueko0/users/nmishra/multiling_fludetection/evals/gridsearch_revisedcateg\n',
 '\n',
 'Number of tweets in data: 4284\n']

In [64]:
training_language = split_list(alist=lines, startswith="Training data used for ")
print(len(training_language))

6


In [65]:
all_df = []
for trained in training_language:
    trained_models = split_list(alist=trained, startswith="{'MODEL_CHECKPOINT':")
    trained_lang = trained[0].split()[-2]
    print(trained_lang)
    for model in trained_models:
        model_runs = split_list(alist=model, startswith="Training")
        for run in model_runs:
            results = [ast.literal_eval(i) for i in run if i.startswith('{')]
            df = pd.DataFrame.from_dict([i for i in results if 'eval_loss' in i.keys()])
            df = df[['eval_loss', 'eval_f1', 'epoch']]
            df['batch_size'] = results[0]['batch_size']
            df['learning_rate'] = results[0]['learning_rate']
            df['model_name'] = run[0].split()[1]
            df['trained_on'] = trained_lang
            all_df.append(df)
all_df = pd.concat(all_df, ignore_index=True)
all_df.head()

all
de
en
es
fr
it


,eval_loss,eval_f1,epoch,batch_size,learning_rate,model_name,trained_on
0,0.292151,0.864522,1.0,32,0.000054,bernice,all
1,0.396538,0.858083,2.0,32,0.000054,bernice,all
2,0.464542,0.881948,3.0,32,0.000054,bernice,all
3,0.572202,0.872443,4.0,32,0.000054,bernice,all
4,0.751824,0.869717,5.0,32,0.000054,bernice,all


In [66]:
all_df.to_csv(LOG_PATH.joinpath(f'{LOG_NUM}.csv'), index=False)

LOAD ALL GRIDSEARCH FILES AND COMBINE

In [85]:
[print(i) for i in LOG_PATH.glob('*.csv')]

/gaueko0/users/nmishra/multiling_fludetection/logs/9350.csv
/gaueko0/users/nmishra/multiling_fludetection/logs/9285.csv


[None, None]

In [86]:
all_df = pd.concat([pd.read_csv(i) for i in LOG_PATH.glob('*.csv')])
all_df.sort_values(['trained_on', 'model_name']).to_csv('all_gridsearch.csv', index=False)

In [87]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1276 entries, 0 to 791
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   eval_loss      1276 non-null   float64
 1   eval_f1        1276 non-null   float64
 2   epoch          1276 non-null   float64
 3   batch_size     1276 non-null   int64  
 4   learning_rate  1276 non-null   float64
 5   model_name     1276 non-null   object 
 6   trained_on     1276 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 79.8+ KB


In [88]:
# for grp, trained_on in all_df.groupby('trained_on'):
#     print('----')
#     print(grp)
#     for model_name, model in trained_on.groupby('model_name'):
#         model.reset_index(inplace=True)
        # print(model_name)
        # print(model)
        # print(model['eval_f1'].max())
        # print(trained_on.groupby('model_name').agg({'eval_f1': 'max', 'eval_loss': 'min'}))
